In [1]:
import os
os.chdir("/app")

In [2]:
import pandas as pd
from pathlib import Path
from nbgrader.api import Gradebook
from nbgrader.apps import NbGraderAPI
from traitlets.config import Config
from tqdm.auto import tqdm
import re
import os
import time

In [3]:
COURSE_ID = "shad_1_2025_august"

In [4]:
config = Config()
config.CourseDirectory.course_id = COURSE_ID
# config.ExecutePreprocessor.environment = {
#     'DATA_PATH_2': '/path/to/data'
# }
config.ExecutePreprocessor.timeout = 1200
config.ExecutePreprocessor.kernel_name="python3"

In [5]:
nbg = NbGraderAPI(config=config)
gradebook = nbg.gradebook

In [6]:
nbg.get_assignments()

[{'id': 'd0cfc9aef0a740189a9b2b0f6117c6bb',
  'name': 'hw_1',
  'duedate': None,
  'num_submissions': 39,
  'max_score': 20.0,
  'max_code_score': 20.0,
  'max_written_score': 0.0,
  'max_task_score': 0.0,
  'display_duedate': None,
  'duedate_notimezone': None,
  'duedate_timezone': '+0000',
  'average_score': 15.564102564102564,
  'average_code_score': 15.564102564102564,
  'average_written_score': 0.0,
  'average_task_score': 0.0,
  'releaseable': True,
  'status': 'released',
  'source_path': 'source/hw_1',
  'release_path': 'release/hw_1'},
 {'id': '596665af706249cf8033fd4d97dbeca3',
  'name': 'hw_2',
  'duedate': None,
  'num_submissions': 19,
  'max_score': 80.0,
  'max_code_score': 80.0,
  'max_written_score': 0.0,
  'max_task_score': 0.0,
  'display_duedate': None,
  'duedate_notimezone': None,
  'duedate_timezone': '+0000',
  'average_score': 58.10526315789474,
  'average_code_score': 58.10526315789474,
  'average_written_score': 0.0,
  'average_task_score': 0.0,
  'releaseab

In [7]:
ASSIGNMENT = "hw_2"
NOTEBOOK = "hse_project"  # without .ipynb
# MAX_SCORE = 20

In [8]:
os.environ["DATA_PATH"] = "/usr/local/share/nbgrader/exchange/data/data_original"

# Collect

In [10]:
nbg.collect(ASSIGNMENT, update=False)

{'success': True,
 'log': "[INFO] Processing 23 submissions of 'hw_2' for course 'shad_1_2025_august'\n[WARNING] \n    /usr/local/share/nbgrader/exchange/shad_1_2025_august/inbound/Тюлякова_Ольга+hw_2+2025-09-14T17:25:39 UTC+00 claims to be submitted by Тюлякова_Ольга but is owned by dev; cheating attempt?\n    you may disable this warning by unsetting the option CollectApp.check_owner\n    \n[INFO] Collecting submission: Тюлякова_Ольга hw_2\n[WARNING] \n    /usr/local/share/nbgrader/exchange/shad_1_2025_august/inbound/Рукавица_Артём+hw_2+2025-08-31T21:27:07 UTC+00 claims to be submitted by Рукавица_Артём but is owned by dev; cheating attempt?\n    you may disable this warning by unsetting the option CollectApp.check_owner\n    \n[INFO] Submission already exists, use --update to update: Рукавица_Артём hw_2\n[WARNING] \n    /usr/local/share/nbgrader/exchange/shad_1_2025_august/inbound/Жданов_Алексей+hw_2+2025-09-14T17:25:39 UTC+00 claims to be submitted by Жданов_Алексей but is owned by

# Autograde

In [11]:
submitted_student_ids = nbg.get_submitted_students(ASSIGNMENT)
type(submitted_student_ids), len(submitted_student_ids)

(set, 23)

In [12]:
autograded_student_ids = nbg.get_autograded_students(ASSIGNMENT)
type(autograded_student_ids), len(autograded_student_ids)

(set, 18)

In [13]:
not_autograded_student_ids = list(submitted_student_ids - autograded_student_ids)
len(not_autograded_student_ids)

5

In [14]:
not_autograded_student_ids

['Тюлякова_Ольга',
 'Шестакова_Анна',
 'Таскаев_Владислав',
 'Жданов_Алексей',
 'Гришкин_Иван']

In [15]:
manually_graded_student_ids = {
    "Сухарев_Денис",
    "Шестакова_Анна",
}

In [16]:
# student_ids_requested_to_grade = None
# student_ids_requested_to_grade = submitted_student_ids
student_ids_requested_to_grade = list(set(not_autograded_student_ids) - manually_graded_student_ids)

In [17]:
if student_ids_requested_to_grade is not None:
    student_ids_to_grade = student_ids_requested_to_grade
    force = True
else:
    student_ids_to_grade = not_autograded_student_ids
    force = False

In [18]:
f"{force=}", student_ids_to_grade

('force=True',
 ['Таскаев_Владислав', 'Тюлякова_Ольга', 'Гришкин_Иван', 'Жданов_Алексей'])

In [19]:
results = {}
for student_id in tqdm(student_ids_to_grade):
    ts = time.time()
    res = nbg.autograde(assignment_id=ASSIGNMENT, student_id=student_id, force=force, create=True)
    elapsed = time.time() - ts
    res["time"] = elapsed
    results[student_id] = res

  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/site-packages/nbgrader/api.py:2255: SAWarning: Object of type <SubmittedAssignment> not in session, add operation along 'Assignment.submissions' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happened while initializing objects.)
  for notebook in submission.assignment.notebooks:
/usr/local/lib/python3.10/site-packages/nbgrader/api.py:2255: SAWarning: Object of type <SubmittedAssignment> not in session, add operation along 'Student.submissions' will not proceed (This warning originated from the Session 'autoflush' process, which was invoked automatically in response to a user-initiated operation. Consider using ``no_autoflush`` context manager if this warning happened while initializing objects.)
  for notebook in submission.assignment.notebooks:
/usr/local/lib/python3.10/site-packages/nbgra

In [20]:
successes = {sid: res for sid, res in results.items() if res["success"]}
failures = {sid: res for sid, res in results.items() if not res["success"]}
len(successes), len(failures)

(4, 0)

In [16]:
print(list(failures.values())["log"])

TypeError: list indices must be integers or slices, not str

# Get per-task scores

In [21]:
student_ids_requested_to_grade = None

In [22]:
if student_ids_requested_to_grade is not None:
    students_to_get_scores = student_ids_requested_to_grade
else:
    students_to_get_scores = nbg.get_autograded_students(ASSIGNMENT)

In [24]:
students_to_get_scores = list(successes.keys())
students_to_get_scores

['Таскаев_Владислав', 'Тюлякова_Ольга', 'Гришкин_Иван', 'Жданов_Алексей']

In [25]:
scores = {}
for student_id in tqdm(students_to_get_scores):
    nb = gradebook.find_submission_notebook(NOTEBOOK, ASSIGNMENT, student_id)
    student_scores = {
        "total": {"actual": nb.score, "max": nb.max_score},
        "cells": [
            {"actual": g.score, "max": g.max_score}
            for g in nb.grades
        ]
    }
    scores[student_id] = student_scores

  0%|          | 0/4 [00:00<?, ?it/s]

In [26]:
total_scores = {sid: int(s["total"]["actual"]) for sid, s in scores.items()}
len(total_scores)

4

In [27]:
total_scores

{'Таскаев_Владислав': 44,
 'Тюлякова_Ольга': 60,
 'Гришкин_Иван': 8,
 'Жданов_Алексей': 80}

In [28]:
scores

{'Таскаев_Владислав': {'total': {'actual': 44.0, 'max': 80.0},
  'cells': [{'actual': 0.0, 'max': 16.0},
   {'actual': 16.0, 'max': 16.0},
   {'actual': 8.0, 'max': 8.0},
   {'actual': 20.0, 'max': 20.0},
   {'actual': 0.0, 'max': 20.0}]},
 'Тюлякова_Ольга': {'total': {'actual': 60.0, 'max': 80.0},
  'cells': [{'actual': 16.0, 'max': 16.0},
   {'actual': 16.0, 'max': 16.0},
   {'actual': 8.0, 'max': 8.0},
   {'actual': 20.0, 'max': 20.0},
   {'actual': 0.0, 'max': 20.0}]},
 'Гришкин_Иван': {'total': {'actual': 8.0, 'max': 80.0},
  'cells': [{'actual': 0.0, 'max': 16.0},
   {'actual': 0.0, 'max': 16.0},
   {'actual': 8.0, 'max': 8.0},
   {'actual': 0.0, 'max': 20.0},
   {'actual': 0.0, 'max': 20.0}]},
 'Жданов_Алексей': {'total': {'actual': 80.0, 'max': 80.0},
  'cells': [{'actual': 16.0, 'max': 16.0},
   {'actual': 16.0, 'max': 16.0},
   {'actual': 8.0, 'max': 8.0},
   {'actual': 20.0, 'max': 20.0},
   {'actual': 20.0, 'max': 20.0}]}}

# Make human-readable feedback

In [29]:
CELL_NAMES = [
    "ImplicitALS",
    "SVD",
    "dataset_with_features",
    "ImplicitALS_with_features",
    "LightFM_with_features",
]

**Скоры предполагаются целыми**

In [30]:
feedbacks = []
for student_id, student_scores in scores.items():
    total_score = student_scores["total"]
    feedback_scores = [f"total: {total_score['actual']:.0f} / {total_score['max']:.0f}"]
    for cell_name, cell_score in zip(CELL_NAMES, student_scores["cells"]):
        feedback_scores.append(f"{cell_name}: {cell_score['actual']:.0f} / {cell_score['max']:.0f}")
    feedback = {
        "student_id": student_id,
        "feedback": ", ".join(feedback_scores),
        "score": round(total_score["actual"])
    }
    feedbacks.append(feedback)

In [31]:
feedbacks

[{'student_id': 'Таскаев_Владислав',
  'feedback': 'total: 44 / 80, ImplicitALS: 0 / 16, SVD: 16 / 16, dataset_with_features: 8 / 8, ImplicitALS_with_features: 20 / 20, LightFM_with_features: 0 / 20',
  'score': 44},
 {'student_id': 'Тюлякова_Ольга',
  'feedback': 'total: 60 / 80, ImplicitALS: 16 / 16, SVD: 16 / 16, dataset_with_features: 8 / 8, ImplicitALS_with_features: 20 / 20, LightFM_with_features: 0 / 20',
  'score': 60},
 {'student_id': 'Гришкин_Иван',
  'feedback': 'total: 8 / 80, ImplicitALS: 0 / 16, SVD: 0 / 16, dataset_with_features: 8 / 8, ImplicitALS_with_features: 0 / 20, LightFM_with_features: 0 / 20',
  'score': 8},
 {'student_id': 'Жданов_Алексей',
  'feedback': 'total: 80 / 80, ImplicitALS: 16 / 16, SVD: 16 / 16, dataset_with_features: 8 / 8, ImplicitALS_with_features: 20 / 20, LightFM_with_features: 20 / 20',
  'score': 80}]